In this project, you will record 200 images of your face and train a face recognier classifier to recognize your face.  The classifier will inform you if it has detected your face or not!

In [37]:
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join

Note: for each step you will need to specify your appropirate paths of where you store the HAAR files, and the facial images, upon capturing them with your webcam.  I use 'myface/'.  You can also create these subfolder and save and retrieve your images there.

### Load HAAR face classifier

In [38]:
face_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')
print("HAAR face classifier loaded successfully")

HAAR face classifier loaded successfully


The following function uses the face classifier to detect the face, extracts, converts the images to greyscale and resizes them. Ot retirns a croped face image 

In [39]:
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

### Create the data (your face)

The next block of code initializes the webcam, and collects your face pictures from your webcam, until 200 pictures are recorded.   During recording, you should attempt to move your face around to capture front and side profiles and other variations.  This will improve the classification step and enhance the chances of detecting your face in the test phase.

When the recording reaches 200, select Enter to exit the webcam view.

In [47]:
# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = 'myface/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 200: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

### Train the classifier

In the next block of code, we will retrieve the training data we just created, create a numpy array of the training images, append the label for each image (based on index), and also create a numpy array for the labels

In [43]:
# Get the training data we previously made
data_path = 'myface/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)



Initalizing the facial recongizer and training our model

In [44]:
# Initialize facial recognizer
#model = cv2.createLBPHFaceRecognizer()
model =cv2.face.LBPHFaceRecognizer_create()

# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()

# Let's train our model 
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")


Model trained sucessefully


### Run and test the Facial Recognizer

The following function perfomrs the following steps: <br>
- grayscales the frames of the webcam
- uses the HAAR face classifier declared above to detect the face.

When a face si detected: <br>
- a rectangle outline is placed over the face
- it is resized too 200, 200 to match input 

Returns the image and the corrected image   


In [45]:
def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi

The following block of code opens the webcam, reads each frame, detects the face, perform the processing (convert to greyscale, resizes, etc),  and performs prediction based on previous training step using the LBPHFaceRecognizer.

The confidence percetnage will be printed to the webcam screen.
If the recognition classifer is above the threshold, "It's you!!!" will appear in green.
If the recognition falls below the threshold, the word "Sorry - this isn't you!" will appreas in red

Hit Enter to exit the webcam view.

Note: To test, load a pic of someone else on you mobile phone and hold it up to the webcam (I did Micehle Obama) and see if the program ramains locked.  Good luck!

In [46]:
# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 75:
            cv2.putText(image, "It's you!!!", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
        else:
            cv2.putText(image, "Sorry - this isn't you!", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "Could not detect a face", (120, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "Not clear", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()  

Tutorial from Mastering Computer vision with Pyhton and OpenCV